In [1]:
import os
import json
import random
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import warnings
warnings.filterwarnings('ignore')

from net import weights_init, Generator, Discriminator

In [2]:
SETTING_PATH = "./parameters.json"

with open(SETTING_PATH) as f:
    parameters = json.load(f)

In [3]:
workers = int(parameters["workers"])
batch_size = int(parameters["batch_size"])
nz = int(parameters["nz"])
nch_g = int(parameters["image_size"])
nch_d = int(parameters["image_size"])
image_size = int(parameters["image_size"])
n_epoch = int(parameters["num_epochs"])
lr = float(parameters["lr"])
beta1 = float(parameters["beta"])
outf = parameters["output_dir"]
display_interval = 100

try:
    os.makedirs(outf)
except OSError:
    pass

seed_num = parameters["seed_num"]

random.seed(seed_num)
np.random.seed(seed_num)
torch.manual_seed(seed_num)

In [4]:
train_size = int(parameters["subset_size"])
!pwd

/workspace/work


In [6]:
dataroot = "./img/"

# We can use an image folder dataset the way we have it setup.
# Create the dataset
testset = dset.STL10(root='./dataset/stl10_root', download=True, split='test',
                     transform=transforms.Compose([
                         transforms.RandomResizedCrop(64, scale=(88/96, 1.0), ratio=(1., 1.)),
                         transforms.RandomHorizontalFlip(),
                         transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                     ]))

dataset = testset


0it [00:00, ?it/s]


  1%|          | 23552000/2640397119 [00:15<1:32:54, 469404.82it/s]


  1%|▏         | 35782656/2640397119 [00:29<34:06, 1272858.70it/s]


  2%|▏         | 50683904/2640397119 [00:43<1:26:51, 496918.16it/s]


  2%|▏         | 60424192/2640397119 [00:56<43:48, 981708.60it/s]


...

 98%|█████████▊| 2597421056/2640397119 [23:37<00:15, 2833221.21it/s]


100%|█████████▉| 2632548352/2640397119 [23:50<00:04, 1685505.87it/s]


100%|█████████▉| 2640273408/2640397119 [23:54<00:00, 1782668.97it/s]

Extracting ./dataset/stl10_root/stl10_binary.tar.gz to ./dataset/stl10_root



2640404480it [24:05, 1782668.97it/s]                                

In [7]:
from torch.utils.data.dataset import Subset

train_size = int(parameters["subset_size"])
subset1_indices = list(range(0,train_size))
dataset = Subset(dataset, subset1_indices)

In [8]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True)
# 学習に使用するデバイスを得る。可能ならGPUを使用する
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

device: cuda:0


In [9]:
# 生成器G。ランダムベクトルから贋作画像を生成する
netG = Generator(nz=nz, nch_g=nch_g).to(device)
netG.apply(weights_init)    # weights_init関数で初期化
print(netG)

Generator(
  (layers): ModuleDict(
    (layer0): Sequential(
      (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer1): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer2): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer3): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer4): Sequential(
      (0): ConvTranspose2d(64, 3, ker

In [10]:
# 識別器D。画像が、元画像か贋作画像かを識別する
netD = Discriminator(nch_d=nch_d).to(device)
netD.apply(weights_init)
print(netD)

Discriminator(
  (layers): ModuleDict(
    (layer0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2)
    )
    (layer1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer4): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
  )
)


In [11]:
criterion = nn.MSELoss()    # 損失関数は平均二乗誤差損失

# オプティマイザ−のセットアップ
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)  # 識別器D用
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)  # 生成器G用

fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)  # 確認用の固定したノイズ

In [12]:
from tensorboardX import SummaryWriter

# Writer will output to ./runs/ directory by default
log_path = "./logs"
writer = SummaryWriter(log_path)

In [13]:
img_list = []
G_losses = np.array([])
D_losses = np.array([])
iters = 0


# 学習のループ
for epoch in range(n_epoch):
    for itr, data in enumerate(dataloader):
        real_image = data[0].to(device)     # 元画像
        sample_size = real_image.size(0)    # 画像枚数
        noise = torch.randn(sample_size, nz, 1, 1, device=device)   # 正規分布からノイズを生成
        
        real_target = torch.full((sample_size,), 1., device=device)     # 元画像に対する識別信号の目標値「1」
        fake_target = torch.full((sample_size,), 0., device=device)     # 贋作画像に対する識別信号の目標値「0」
        
        ############################
        # 識別器Dの更新
        ###########################
        netD.zero_grad()    # 勾配の初期化

        output = netD(real_image)   # 識別器Dで元画像に対する識別信号を出力
        errD_real = criterion(output, real_target)  # 元画像に対する識別信号の損失値
        
        D_x = output.mean().item()

        fake_image = netG(noise)    # 生成器Gでノイズから贋作画像を生成
        
        output = netD(fake_image.detach())  # 識別器Dで元画像に対する識別信号を出力
        errD_fake = criterion(output, fake_target)  # 贋作画像に対する識別信号の損失値
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake    # 識別器Dの全体の損失
        errD.backward()    # 誤差逆伝播
        optimizerD.step()   # Dのパラメーターを更新

        ############################
        # 生成器Gの更新
        ###########################
        netG.zero_grad()    # 勾配の初期化
        
        output = netD(fake_image)   # 更新した識別器Dで改めて贋作画像に対する識別信号を出力
        errG = criterion(output, real_target)   # 生成器Gの損失値。Dに贋作画像を元画像と誤認させたいため目標値は「1」
        errG.backward()     # 誤差逆伝播
        D_G_z2 = output.mean().item()

        optimizerG.step()   # Gのパラメータを更新

        if itr % display_interval == 0: 
            print('[{}/{}][{}/{}] Loss_D: {:.3f} Loss_G: {:.3f} D(x): {:.3f} D(G(z)): {:.3f}/{:.3f}'
                  .format(epoch + 1, n_epoch,
                          itr + 1, len(dataloader),
                          errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if epoch == 0 and itr == 0:     # 初回に元画像を保存する
            vutils.save_image(real_image, '{}/real_samples.png'.format(outf),
                              normalize=True, nrow=10)
            
        # Save Losses for plotting later
        G_losses = np.append(G_losses,errG.item())
        D_losses = np.append(D_losses,errD.item())
        #G_losses.append(errG.item())
        #D_losses.append(errD.item())
        
        # Tensorboard用のデータ
        writer.add_scalar("G_loss",errG.item(),iters)    
        writer.add_scalar("D_loss",errD.item(),iters)
        
        iters += 1
        
    
    ############################
    # 確認用画像の生成
    ############################
    fake_image = netG(fixed_noise)  # 1エポック終了ごとに確認用の贋作画像を生成する
#     vutils.save_image(fake_image.detach(), '{}/fake_samples_epoch_{:03d}.png'.format(outf, epoch + 1),
#                       normalize=True, nrow=10)
    writer.add_image("fake_image",vutils.make_grid(fake_image), epoch)

    ############################
    # モデルの保存
    ############################
    if (epoch + 1) % 50 == 0:   # 50エポックごとにモデルを保存する
        torch.save(netG.state_dict(), '{}/netG_epoch_{}.pth'.format(outf, epoch + 1))
        torch.save(netD.state_dict(), '{}/netD_epoch_{}.pth'.format(outf, epoch + 1))

[1/200][1/8] Loss_D: 4.366 Loss_G: 50.139 D(x): 0.695 D(G(z)): 1.375/-6.041
[2/200][1/8] Loss_D: 6.605 Loss_G: 22.133 D(x): 0.590 D(G(z)): 1.805/-3.600
[3/200][1/8] Loss_D: 3.945 Loss_G: 55.882 D(x): 0.602 D(G(z)): 1.306/-6.406
[4/200][1/8] Loss_D: 2.392 Loss_G: 88.293 D(x): 0.728 D(G(z)): 0.363/-8.364
[5/200][1/8] Loss_D: 1.421 Loss_G: 114.059 D(x): 0.857 D(G(z)): -0.087/-9.657
[6/200][1/8] Loss_D: 6.385 Loss_G: 32.593 D(x): 0.575 D(G(z)): 2.192/-4.689
[7/200][1/8] Loss_D: 1.726 Loss_G: 31.953 D(x): 0.689 D(G(z)): -0.929/-4.640
[8/200][1/8] Loss_D: 1.263 Loss_G: 68.811 D(x): 0.663 D(G(z)): 0.447/-7.284
[9/200][1/8] Loss_D: 3.478 Loss_G: 10.899 D(x): 0.345 D(G(z)): -1.578/-2.290
[10/200][1/8] Loss_D: 2.067 Loss_G: 19.355 D(x): 1.141 D(G(z)): 1.270/-3.390
[11/200][1/8] Loss_D: 0.478 Loss_G: 15.008 D(x): 0.855 D(G(z)): -0.019/-2.866
[12/200][1/8] Loss_D: 0.749 Loss_G: 24.398 D(x): 0.928 D(G(z)): 0.527/-3.930
[13/200][1/8] Loss_D: 2.268 Loss_G: 29.817 D(x): 0.736 D(G(z)): 1.346/-4.454
[14

KeyboardInterrupt: 

In [14]:
import tensorflow as tf

event_file_path = 'logs/events.out.tfevents.1567349477.30fe1aa1fb2d'

for event in tf.train.summary_iterator(event_file_path):
    event_type = event.WhichOneof('what')
    if event_type != 'summary':
        writer.add_event(event)
    else:
        wall_time = event.wall_time
        step = event.step

        # possible types: simple_value, image, histo, audio
        filtered_values = [value for value in event.summary.value if value.HasField('image')]
        summary = tf.Summary(value=filtered_values)

        filtered_event = tf.summary.Event(summary=summary,
                                          wall_time=wall_time,
                                          step=step)
        writer.add_event(filtered_event)


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

AttributeError: 'SummaryWriter' object has no attribute 'add_event'

In [ ]:


for event in tf.train.summary_iterator('logs/events.out.tfevents.1567345099.30fe1aa1fb2d'):
    for value in event.summary.value:
        print(value.tag)
        if value.HasField('simple_value'):
            print(value.simple_value)